In [2]:
!nvidia-smi

Thu Jun 24 09:51:06 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
# !mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [5]:
!pip install kaggle -q
!kaggle datasets download -d aladdinpersson/flickr8kimagescaptions
!unzip -q flickr8kimagescaptions.zip

 98% 1.01G/1.04G [00:06<00:00, 154MB/s]
100% 1.04G/1.04G [00:06<00:00, 163MB/s]


In [32]:
!rm -r Image-Captioning/

In [33]:
# # get the code for kaggle
!git clone https://github.com/moaaztaha/Image-Captioning
py_files_path = 'Image-Captioning/'
import sys
sys.path.append(py_files_path)

Cloning into 'Image-Captioning'...
remote: Enumerating objects: 526, done.
remote: Counting objects: 100% (526/526), done.
remote: Compressing objects: 100% (233/233), done.
remote: Total 526 (delta 320), reused 491 (delta 285), pack-reused 0
Receiving objects: 100% (526/526), 36.81 MiB | 23.08 MiB/s, done.
Resolving deltas: 100% (320/320), done.


In [34]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [35]:
# !pip3 install nltk==3.6.2
# import nltk
# nltk.download("wordnet")

In [36]:
import time 
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torch import nn
from torch.nn.utils.rnn import pack_padded_sequence
from models import Encoder, DecoderWithAttention
from dataset import *
from utils import *
from train import *
from torch.utils.tensorboard import SummaryWriter
from os import path as osp

In [43]:
# Model parameters
encoder_dim = 2048 # resnet101
emb_dim = 512  # dimension of word embeddings
attention_dim = 512  # dimension of attention linear layers
decoder_dim = 512  # dimension of decoder RNN
dropout = 0.5
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # sets device for model and PyTorch tensors
cudnn.benchmark = True  # set to true only if inputs to model are fixed size; otherwise lot of computational overhead

# training parameters
epochs = 30  # number of epochs to train for (if early stopping is not triggered)
batch_size = 256
workers = 2
encoder_lr = 1e-4  # learning rate for encoder if fine-tuning
decoder_lr = 4e-4  # learning rate for decoder
fine_tune_encoder = False  # fine-tune encoder?
pretrained_embeddings = False
fine_tune_embeddings = False
checkpoint = None  # path to checkpoint, None if none

In [48]:
DATA_NAME = 'flickr8k_10k'

# local
# DATA_JSON_PATH = 'data.json'
# IMGS_PATH = 'flickr/Images/'
# kaggle paths
# DATA_JSON_PATH = '/kaggle/working/Image-Captioning/data.json'
# IMGS_PATH = '../input/flickr8kimagescaptions/flickr8k/images/'
#colab
DATA_JSON_PATH = 'Image-Captioning/data.json'
IMGS_PATH = 'flickr8k/images/'

In [49]:
# load vocab
# vocab = build_vocab(DATA_JSON_PATH)
top10k_words = get_10k_vocab("/content/Image-Captioning/10k_words.txt")
vocab = top10k_vocab(top10k_words)
vocab_len = len(vocab)
vocab_len

100%|██████████| 1/1 [00:00<00:00, 123.54it/s]


10004

In [64]:
t_params = {
    'data_name': DATA_NAME,
    'imgs_path': IMGS_PATH,
    'df_path': DATA_JSON_PATH,
    'vocab': vocab,
    'epochs': epochs,
    'batch_size': batch_size,
    'workers': workers,
    'decoder_lr': decoder_lr,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder,
    'pretrained_embeddings': pretrained_embeddings,
}

m_params = {
    'attention_dim': attention_dim,
    'embed_dim': emb_dim,
    'decoder_dim': decoder_dim,
    'encoder_dim': encoder_dim,
    'dropout': dropout
}

logger_dic = {
    'decoder_lr': decoder_lr,
    'encoder_lr': encoder_lr,
    'fine_tune_encoder': fine_tune_encoder,
    'pretrained_embeddings': pretrained_embeddings,
    'max_seq_length': 100,
    'vocab_size': vocab_len,
    'enocder': 'resnet101',
    'dropout': dropout,
    'attention_dim': attention_dim,
    'embed_dim': emb_dim,
    'decoder_dim': decoder_dim,
    'encoder_dim': encoder_dim 
    
}

In [57]:
t_params

{'batch_size': 64,
 'data_name': 'flickr8k_10k',
 'decoder_lr': 0.0004,
 'df_path': 'Image-Captioning/data.json',
 'encoder_lr': 0.0001,
 'epochs': 30,
 'fine_tune_encoder': True,
 'imgs_path': 'flickr8k/images/',
 'pretrained_embeddings': False,
 'vocab': <dataset.Vocabulary at 0x7fd41be8fdd0>,
 'workers': 2}

In [52]:
# experiment name
name = DATA_NAME + "_10k_words"
# path
log_dir = '/content/drive/MyDrive/ImageCaptioning/flickr8/experiments'

logger = SummaryWriter(log_dir=osp.join(log_dir, name))

In [53]:
# with scheduler
fit(t_params=t_params, m_params=m_params, logger=logger)

Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth



Loading Data
Dataset split: train
Unique images: 6000
Total size: 30000
Dataset split: val
Unique images: 1000
Total size: 5000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [0][0/118]	Batch Time 9.145 (9.145)	Data Load Time 3.795 (3.795)	Loss 10.1185 (10.1185)	Top-5 Accuracy 0.000 (0.000)
Epoch: [0][100/118]	Batch Time 2.504 (2.568)	Data Load Time 0.007 (0.042)	Loss 5.0194 (5.6171)	Top-5 Accuracy 51.929 (44.116)
Epoch train time 301.370 (epoch_time.avg:.3f)
-------------------- Validation --------------------
Validation: [0/20]	Batch Time 6.024 (6.024)	Loss 5.5496 (5.5496)	Top-5 Accuracy 50.943 (50.943)	
----- Bleu-n Scores -----
1: 60.1798427021706
2: 35.488586163035684
3: 18.514052756816376
4: 9.881429249562874
-------------------------

 * LOSS - 5.520, TOP-5 ACCURACY - 50.609, BLEU-4 - 9.881429249562874

Epoch validation time 55.042 (epoch_time.avg:.3f)
__________________________________________________
-------------------- Training --------------------
Epoch: [1][0/118]	Batch Time 6.455 (6.455)	Data Load Time 3.543 (3.543)	Loss 4.7334 (4.7334)	Top-5 Accuracy 55.526 (55.526)
Epoch: [1][100/118]	Batch Time 2.544 (2.581)	Data Load Time 0.012 (0.0

In [65]:
batch_size = 64
fine_tune_encoder = True
checkpoint = '/content/BEST_checkpoint_flickr8k_10k.pth.tar'
# epochs = 30

t_params['batch_size'] = batch_size
t_params['data_name'] = t_params['data_name'] + "_finetune" 
t_params['fine_tune_encoder'] = True
t_params['decoder_lr'] = t_params['decoder_lr'] / 10
# t_params['epochs'] = epochs
t_params

{'batch_size': 64,
 'data_name': 'flickr8k_10k_finetune',
 'decoder_lr': 4e-05,
 'df_path': 'Image-Captioning/data.json',
 'encoder_lr': 0.0001,
 'epochs': 30,
 'fine_tune_encoder': True,
 'imgs_path': 'flickr8k/images/',
 'pretrained_embeddings': False,
 'vocab': <dataset.Vocabulary at 0x7fd41be8fdd0>,
 'workers': 2}

In [66]:
m = load_checkpoint(checkpoint)

Loaded Checkpoint!!
Last Epoch: 10
Best Bleu-4: 15.01433469286676


In [67]:
fit(t_params, checkpoint=checkpoint, m_params=m_params, logger=logger)

Loaded Checkpoint!!
Starting Epoch: 11
Loading Data
Dataset split: train
Unique images: 6000
Total size: 30000
Dataset split: val
Unique images: 1000
Total size: 5000
__________________________________________________
-------------------- Fitting --------------------
__________________________________________________
-------------------- Training --------------------
Epoch: [11][0/469]	Batch Time 4.717 (4.717)	Data Load Time 0.994 (0.994)	Loss 4.0034 (4.0034)	Top-5 Accuracy 65.419 (65.419)
Epoch: [11][100/469]	Batch Time 1.339 (1.393)	Data Load Time 0.001 (0.011)	Loss 3.8959 (3.9596)	Top-5 Accuracy 67.725 (65.133)
Epoch: [11][200/469]	Batch Time 1.379 (1.372)	Data Load Time 0.001 (0.006)	Loss 3.8045 (3.9352)	Top-5 Accuracy 66.024 (65.701)
Epoch: [11][300/469]	Batch Time 1.323 (1.366)	Data Load Time 0.002 (0.005)	Loss 3.6938 (3.9122)	Top-5 Accuracy 69.464 (66.075)
Epoch: [11][400/469]	Batch Time 1.345 (1.363)	Data Load Time 0.001 (0.004)	Loss 3.7101 (3.8970)	Top-5 Accuracy 68.865 (66.29

In [69]:
!cp BEST_checkpoint_flickr8k_10k_finetune.pth.tar /content/drive/MyDrive/ImageCaptioning/flickr8 

In [72]:
checkpoint = load_checkpoint("BEST_checkpoint_flickr8k_10k_finetune.pth.tar")
decoder = checkpoint['decoder']
decoder = decoder.to(device)
decoder.eval()
encoder = checkpoint['encoder']
encoder = encoder.to(device)
encoder.eval();

Loaded Checkpoint!!
Last Epoch: 11
Best Bleu-4: 15.591898806714797


In [73]:
from eval import test_score

test_dict = {}

for i in [1, 3, 5]:
    
    b1, b2, b3, b4 = test_score(i, encoder, decoder, IMGS_PATH, DATA_JSON_PATH, vocab)
    if i == 3:
        test_dict['b1'] = b1
        test_dict['b2'] = b2
        test_dict['b3'] = b3
    
    test_dict[f'b4-b{i}'] = b4

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
EVALUATING AT BEAM SIZE 1:   0%|          | 0/5000 [00:00<?, ?it/s]

Dataset split: test
Unique images: 1000
Total size: 5000


/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
EVALUATING AT BEAM SIZE 1: 100%|██████████| 5000/5000 [02:56<00:00, 28.31it/s]


----- Bleu-n Scores -----
1: 58.17601701675087
2: 41.224822354462354
3: 28.000938478839238
4: 18.72495762865639
-------------------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
EVALUATING AT BEAM SIZE 3:   0%|          | 0/5000 [00:00<?, ?it/s]

...

----- Bleu-n Scores -----
1: 61.8241965973535
2: 44.249082940660614
3: 30.579058561977185
4: 20.87097692129813
-------------------------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
EVALUATING AT BEAM SIZE 5:   0%|          | 0/5000 [00:00<?, ?it/s]

Dataset split: test
Unique images: 1000
Total size: 5000


EVALUATING AT BEAM SIZE 5: 100%|██████████| 5000/5000 [04:01<00:00, 20.71it/s]


----- Bleu-n Scores -----
1: 63.51175587793897
2: 45.608191049287484
3: 31.757167505838897
4: 21.785908024986437
-------------------------
